In [125]:
import sys


In [126]:
OPENAI_API_KEY = "needed for using the model"


In [127]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [128]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_magic_prompt),
])

In [129]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain_solution = prompt | llm | output_parser

In [130]:
system_prompt = """"""
my_magic_prompt = """{translation}"""

In [131]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_magic_prompt),
])

In [132]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain_solution = prompt | llm | output_parser

In [133]:
import pandas as pd

test = pd.read_csv('train_with_translation.csv')

In [134]:
sample = test.translation.sample()
sample = sample.values[0]
sample

'To service an international seminar, a team of translators needs to be assembled. Information about the candidates is presented in the table.\n\n\\begin{tabularx}{\\textwidth}{|C|C|>{\\centering\\arraybackslash}X|} \\hline\nTranslator Number & Languages & Service Cost \\newline (RUB per day) \\\\ \\hline\n1 & Spanish & 2850 \\\\ \\hline\n2 & Spanish, English & 6050 \\\\ \\hline\n3 & English & 1900 \\\\ \\hline\n4 & Spanish, French & 5950 \\\\ \\hline\n5 & French, German & 6800 \\\\ \\hline\n6 & German & 4050 \\\\ \\hline\n\\end{tabularx}\n\\vspace{\\medskip}\n\nUsing the table, assemble at least one group of translators who collectively speak all four languages: English, German, Spanish, and French, and whose total service cost does not exceed 12,000 RUB per day. In your answer, indicate any one set of translator numbers without spaces, commas, or other additional symbols.'

In [135]:
print(my_magic_prompt)

{translation}


In [136]:
solution = chain_solution.invoke({"translation": sample})

In [137]:
solution

'To assemble a group of translators who collectively speak all four languages and whose total service cost does not exceed 12,000 RUB per day, we can select translators 2 and 5.\n\nTranslator 2 speaks Spanish and English, and translator 5 speaks French and German. Together, they cover all four languages required. The total service cost for this group is 6050 + 6800 = 12,850 RUB per day, which exceeds the budget.\n\nTo reduce the cost, we can replace translator 5 with translator 4, who also speaks Spanish. This way, the group consists of translators 2 and 4.\n\nTranslator 2 speaks Spanish and English, and translator 4 speaks Spanish and French. Together, they cover all four languages required. The total service cost for this group is 6050 + 5950 = 12,000 RUB per day, which is within the budget.\n\nTherefore, the set of translator numbers for the group is 24.'

In [169]:
extract_answer_prompt = """Task:
{problem_text}
Solution:
{solution}
Extract the answer according to the task,in the end, please verify your answer using code interpreter by yourself.
output the answer value ONLY, in the same order appearing in the task. Each test answer should be a number — either an integer or finite decimal float with "." as a separator.
Incorrect format examples:
4 2 3 1 - should be printed as 4231
-12.0 - should be printed as 12.
3/4 - should be printed as 0.75

"""

In [170]:
prompt_extract = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", extract_answer_prompt),
])
chain_extract = prompt_extract | llm | output_parser

In [171]:
answer = chain_extract.invoke({
    "problem_text": sample,
    "solution": solution,
})
answer

'136/85'

In [172]:
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm

tqdm.pandas()
test[['answer', 'solution']] = None

def get_answer_llm(translation, chain_solution=chain_solution, chain_extract=chain_extract):
    solution = chain_solution.invoke({
        "translation": translation
    })
    answer = chain_extract.invoke({
        "problem_text": translation,
        "solution": solution,
    })
    return pd.Series({'answer': answer, 'solution': solution})

with get_openai_callback() as cb:
    test.loc[test['answer'].isna(), ['answer', 'solution']] = (
        test.loc[test['answer'].isna(), 'translation'].sample(3).progress_apply(get_answer_llm)
    )
    print(cb)

100%|█████████████████████████████████████████████| 3/3 [00:29<00:00,  9.69s/it]

Tokens Used: 1822
	Prompt Tokens: 1386
	Completion Tokens: 436
Successful Requests: 6
Total Cost (USD): $0.002951


In [173]:

test[~test.answer.isna()]

,problem_id,problem_text,answer,hint,translation,solution
1,4723,В компании из 30 человек 25 пользуются социаль...,12,Диаграмма Венна,"In a company of 30 people, 25 use the social n...",The statements that are true under the given c...
56,8269,Выпускники 11 <<А>> покупают букеты цветов для...,1680,подсчеты,"The graduates of class 11 ""A"" are buying bouqu...","There are 15 teachers in total, including the ..."
96,7097,Среди всех выпускников школы 10 человек собира...,25,число по проценту,"Among all the school graduates, 10 people are ...","To find the total number of graduates, we need..."


In [174]:
with get_openai_callback() as cb:
    test.loc[test['answer'].isna(), ['answer', 'solution']] = (
        test.loc[test['answer'].isna(), 'problem_text'].progress_apply(get_answer_llm)
    )
    print(cb)

100%|███████████████████████████████████████████| 97/97 [28:27<00:00, 17.60s/it]

Tokens Used: 90656
	Prompt Tokens: 64567
	Completion Tokens: 26089
Successful Requests: 194
Total Cost (USD): $0.1490285


In [175]:
test.head(5)

,problem_id,problem_text,answer,hint,translation,solution
0,2374,Найдите значение выражения $\dfrac{17}{5} :\df...,4.3,Десятичную в обыкновенную,Find the value of the expression $\dfrac{17}{5...,Выполним деление дробей: $\dfrac{17}{5} :\dfra...
1,4723,В компании из 30 человек 25 пользуются социаль...,12,Диаграмма Венна,"In a company of 30 people, 25 use the social n...",The statements that are true under the given c...
2,7135,Число дорожно-транспортных происшествий (ДТП) ...,32,дробь в проценты,The number of road traffic accidents (RTAs) in...,Для решения этой задачи необходимо вычислить р...
3,5814,Найдите значение выражения $\dfrac{2\strut^{-5...,256,свойства степени,Find the value of the expression $\dfrac{2^{-5...,Для упрощения выражения воспользуемся свойства...
4,9237,Путешественник из Москвы хочет посетить четыре...,43,перебор,A traveler from Moscow wants to visit four cit...,"Один из вариантов маршрутов, который позволит ..."


In [176]:
test[['problem_id', 'answer']].to_csv('benchmark_submission_gpt.csv', index=False)

In [177]:
test.to_csv('benchmark_test_analysis_self_verify.csv')

In [178]:
from IPython.display import display_latex

problem_test_index = 0
sample = test.iloc[problem_test_index]

display_latex(sample.problem_text, raw=True)
print('-' * 80)

display_latex(sample.solution, raw=True)
print('-' * 80)

print(sample.answer)

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
4.3


In [183]:
df = pd.read_csv("benchmark_test_analysis_self_verify.csv")

In [184]:
df.rename(columns={'answer': 'our_answer'}, inplace=True)


In [185]:
real_answers = pd.read_csv('train_with_translation.csv')
real_answers = real_answers[['problem_id',"answer"]]

In [186]:
df_merged = df.merge(real_answers, on='problem_id')

In [187]:
df_merged

,Unnamed: 0,problem_id,problem_text,our_answer,hint,translation,solution,answer
0,0,2374,Найдите значение выражения $\dfrac{17}{5} :\df...,4.3,Десятичную в обыкновенную,Find the value of the expression $\dfrac{17}{5...,Выполним деление дробей: $\dfrac{17}{5} :\dfra...,1.6
1,1,4723,В компании из 30 человек 25 пользуются социаль...,12,Диаграмма Венна,"In a company of 30 people, 25 use the social n...",The statements that are true under the given c...,24
2,2,7135,Число дорожно-транспортных происшествий (ДТП) ...,32,дробь в проценты,The number of road traffic accidents (RTAs) in...,Для решения этой задачи необходимо вычислить р...,32
3,3,5814,Найдите значение выражения $\dfrac{2\strut^{-5...,256,свойства степени,Find the value of the expression $\dfrac{2^{-5...,Для упрощения выражения воспользуемся свойства...,256
4,4,9237,Путешественник из Москвы хочет посетить четыре...,43,перебор,A traveler from Moscow wants to visit four cit...,"Один из вариантов маршрутов, который позволит ...",53
...,...,...,...,...,...,...,...,...
95,95,7064,"В школе французский язык изучают 77~учащихся, ...",385,число по проценту,"In the school, 77 students study the French la...",Давайте обозначим общее число учащихся в школе...,385
96,96,7097,Среди всех выпускников школы 10 человек собира...,25,число по проценту,"Among all the school graduates, 10 people are ...","To find the total number of graduates, we need...",25
97,97,5448,"Вероятность того, что новая шариковая ручка пи...",0.79,1-Р,The probability that a new ballpoint pen write...,"Вероятность того, что ручка пишет хорошо, равн...",0.79
98,98,2293,Найдите значение выражения $2\dfrac{5}{14} +5+...,41.285714285714285,от обыкновенных дробей к десятичным,Find the value of the expression $2\frac{5}{14...,Первым делом приведем дробь $2\dfrac{5}{14}$ к...,8.5


In [188]:
print((df_merged['our_answer'] == df_merged['answer']).sum())


55


In [122]:
filtered_df = df_merged[df_merged['our_answer'] != df_merged['answer']]


In [123]:
filtered_df

,Unnamed: 0,problem_id,problem_text,our_answer,hint,translation,solution,answer
0,0,2374,Найдите значение выражения $\dfrac{17}{5} :\df...,0.43,Десятичную в обыкновенную,Find the value of the expression $\dfrac{17}{5...,Разделим дроби $\dfrac{17}{5}$ и $\dfrac{34}{3...,1.6
4,4,9237,Путешественник из Москвы хочет посетить четыре...,43,перебор,A traveler from Moscow wants to visit four cit...,Путешественник может выбрать следующие маршрут...,53
5,5,10703,В конце четверти Вовочка выписал подряд все св...,1115,разложитьна простые множители,"At the end of the term, Vovochka wrote down al...",Поскольку произведение всех отметок равно 5575...,3
6,6,9621,"Найдите четырёхзначное число, которое в 11 раз...",5818,куб числа делится на 11,Find a four-digit number that is 11 times smal...,Пусть искомое число равно $x$. Тогда по услови...,"[3267, 7744]"
9,9,4405,"Найдите $\tg x$, если $\sin x=\dfrac{6}{\sqrt{...",The answer is $\dfrac{6 \sqrt{61}}{55}$.,теорема пифагора или основное триг тождество,"Find $\tan x$, if $\sin x = \frac{6}{\sqrt{61}...","Используя определение тангенса, $\tg x = \dfra...",1.2
18,18,4965,Установите соответствие между величинами и их ...,2314,сопоставить,Establish the correspondence between the magni...,А) масса таблетки лекарства - 3) 500 мг \\\nБ)...,3412
19,19,7643,Каждому из четырёх неравенств в левом столбце ...,4231,пропотенцировать и сопоставить,Each of the four inequalities in the left colu...,To determine the correspondence between the in...,4213
20,20,10661,Из книги выпало несколько идущих подряд листов...,199,"выпали страницы с нечетной по четную, листов в...",Several consecutive pages fell out of a book. ...,Поскольку номер последней страницы перед выпав...,76
22,22,11878,"Про натуральные числа $A$, $B$ и $C$ известно,...","Possible values for the guessed number are 48,...",разложить числа от 340-2 до +2 на множители и ...,"It is known about the natural numbers $A$, $B$...",Обозначим загаданное число за $x$. Тогда по ус...,57
23,23,3509,Для обслуживания международного семинара необх...,346,перебор,"To service an international seminar, a team of...",Для решения задачи мы можем перебрать все возм...,"[135, 346]"


In [124]:
df_merged

,Unnamed: 0,problem_id,problem_text,our_answer,hint,translation,solution,answer
0,0,2374,Найдите значение выражения $\dfrac{17}{5} :\df...,0.43,Десятичную в обыкновенную,Find the value of the expression $\dfrac{17}{5...,Разделим дроби $\dfrac{17}{5}$ и $\dfrac{34}{3...,1.6
1,1,4723,В компании из 30 человек 25 пользуются социаль...,24,Диаграмма Венна,"In a company of 30 people, 25 use the social n...",Давайте рассмотрим каждое утверждение по отдел...,24
2,2,7135,Число дорожно-транспортных происшествий (ДТП) ...,32,дробь в проценты,The number of road traffic accidents (RTAs) in...,Для определения процентного уменьшения числа Д...,32
3,3,5814,Найдите значение выражения $\dfrac{2\strut^{-5...,256,свойства степени,Find the value of the expression $\dfrac{2^{-5...,Применим правило умножения степеней с одинаков...,256
4,4,9237,Путешественник из Москвы хочет посетить четыре...,43,перебор,A traveler from Moscow wants to visit four cit...,Путешественник может выбрать следующие маршрут...,53
...,...,...,...,...,...,...,...,...
95,95,7064,"В школе французский язык изучают 77~учащихся, ...",385,число по проценту,"In the school, 77 students study the French la...",Пусть общее число учащихся в школе равно Х. \n...,385
96,96,7097,Среди всех выпускников школы 10 человек собира...,25,число по проценту,"Among all the school graduates, 10 people are ...",Пусть общее число выпускников школы равно Х. Т...,25
97,97,5448,"Вероятность того, что новая шариковая ручка пи...",0.79,1-Р,The probability that a new ballpoint pen write...,Для решения данной задачи воспользуемся формул...,0.79
98,98,2293,Найдите значение выражения $2\dfrac{5}{14} +5+...,48/7,от обыкновенных дробей к десятичным,Find the value of the expression $2\frac{5}{14...,Приведем все дроби к общему знаменателю. Замет...,8.5
